In [1]:
import time

import matplotlib.pyplot as plt
import numpy as np
import math
import pybobyqa

import cirq
import qsimcirq
import sympy as sp

import cupy as cp
from cuquantum import custatevec as cusv

from cirqqulacs import QulacsSimulatorGpu

In [2]:
##Qubits on Lattice, Lattice Parameters
qubits = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
Nq = len(qubits)

xx_links = [(1, 14), (2, 13), (12, 19), (5, 10), (6, 9), (17, 22), (18, 21), (15, 16)]
yy_links = [(1, 2), (3, 4), (5, 6), (10, 9), (12, 11), (14, 13), (17, 18), (22, 21)]
yy_links_reversed = [(22, 21), (17, 18), (14, 13), (12, 11), (10, 9), (5, 6), (3, 4), (1, 2)]
zz_links = [(0, 1), (2, 3), (4, 5), (6, 7), (9, 8), (11, 10), (13, 12), (15, 14), (16, 17), (18, 19), (21, 20), (23, 22)]
zz_links_reversed = [(23, 22), (21, 20), (18, 19), (16, 17), (15, 14), (13, 12), (11, 10), (9, 8), (6, 7), (4, 5), (2, 3), (0, 1)]

Jx, Jy, Jz = -1.0/math.sqrt(2), -1.0/math.sqrt(2), -1.0
hX = hY = hZ = 0.05/math.sqrt(3)

In [3]:
#Hamiltonian Construction

#Qubit register for Hamiltonian
qreg = cirq.NamedQubit.range(Nq,prefix="q")

#Constructing Pauli Strings
XX_links_strings = []
for (i,j) in xx_links:
    XX_string = []
    for q in range(Nq):
        if q != i and q != j:
            XX_string.append(cirq.I(qreg[q]))
        else:
            XX_string.append(cirq.X(qreg[q]))
    XX_links_strings.append(XX_string)
    
YY_links_strings = []
for (i,j) in yy_links:
    YY_string = []
    for q in range(Nq):
        if q != i and q != j:
            YY_string.append(cirq.I(qreg[q]))
        else:
            YY_string.append(cirq.Y(qreg[q]))
    YY_links_strings.append(YY_string)

ZZ_links_strings = []
for (i,j) in zz_links:
    ZZ_string = []
    for q in range(Nq):
        if q != i and q != j:
            ZZ_string.append(cirq.I(qreg[q]))
        else:
            ZZ_string.append(cirq.Z(qreg[q]))
    ZZ_links_strings.append(ZZ_string)

X_qubit_strings = []
Y_qubit_strings = []
Z_qubit_strings = []
for k in qubits:
    X_string = []
    Y_string = []
    Z_string = []
    for q in range(Nq):
        if q != k:
            X_string.append(cirq.I(qreg[q]))
            Y_string.append(cirq.I(qreg[q]))
            Z_string.append(cirq.I(qreg[q]))
        else:
            X_string.append(cirq.X(qreg[q]))
            Y_string.append(cirq.Y(qreg[q]))
            Z_string.append(cirq.Z(qreg[q]))
    X_qubit_strings.append(X_string)
    Y_qubit_strings.append(Y_string)
    Z_qubit_strings.append(Z_string)

In [4]:
#Hamiltonian Construction (ctd.)

#Convert Pauli Strings into Products
XX_links_terms = []
for XX_link in XX_links_strings:
    XX_links_terms.append(cirq.PauliString(XX_link))

YY_links_terms = []
for YY_link in YY_links_strings:
    YY_links_terms.append(cirq.PauliString(YY_link))
    
ZZ_links_terms = []
for ZZ_link in ZZ_links_strings:
    ZZ_links_terms.append(cirq.PauliString(ZZ_link))

X_qubit_terms = []
for qubit_string in X_qubit_strings:
    X_qubit_terms.append(cirq.PauliString(qubit_string))

Y_qubit_terms = []
for qubit_string in Y_qubit_strings:
    Y_qubit_terms.append(cirq.PauliString(qubit_string))

Z_qubit_terms = []
for qubit_string in Z_qubit_strings:
    Z_qubit_terms.append(cirq.PauliString(qubit_string))

In [5]:
#Hamiltonian Construction (ctd.)

#Add up all the terms
Hamiltonian = 0
for XX_link in XX_links_terms:
    Hamiltonian += Jx*XX_link
for YY_link in YY_links_terms:
    Hamiltonian += Jy*YY_link
for ZZ_link in ZZ_links_terms:
    Hamiltonian += Jz*ZZ_link
for X_qubit in X_qubit_terms:
    Hamiltonian += hX*X_qubit
for Y_qubit in Y_qubit_terms:
    Hamiltonian += hY*Y_qubit
for Z_qubit in Z_qubit_terms:
    Hamiltonian += hZ*Z_qubit

In [6]:
##Hamiltonian Variational Ansatz Circuit Construction
def HVA_circuit(n_layers):
    global xx_links
    global yy_links
    global yy_links_reversed
    global zz_links
    global zz_links_reversed
    global qubits
    global alpha
    global beta
    global gamma
    global alpha_mag
    global beta_mag
    global gamma_mag
    #Initialize Circuit
    HVA = cirq.Circuit()
    qreg_HVA = cirq.NamedQubit.range(Nq, prefix="q")
    #Declare variational parameters
    alpha = sp.symarray('alpha', n_layers)
    beta = sp.symarray('beta', n_layers)
    gamma = sp.symarray('gamma', n_layers)
    alpha_mag = sp.symarray('alpha_mag', n_layers)
    beta_mag = sp.symarray('beta_mag', n_layers)
    gamma_mag = sp.symarray('gamma_mag', n_layers)
    #Ansatz Construction
    for idx_layer in range(n_layers):
    #Construct X-terms portion of layer
        HVA.append(cirq.rx(2*alpha_mag[idx_layer]).on(q) for q in qreg_HVA)
        for (i,j) in xx_links:
            HVA.append(cirq.H(qreg_HVA[i]))
            HVA.append(cirq.H(qreg_HVA[j]))
            HVA.append(cirq.CNOT(qreg_HVA[max(i,j)],qreg_HVA[min(i,j)]))
            HVA.append(cirq.rz(2*alpha[idx_layer]).on(qreg_HVA[min(i,j)]))
            HVA.append(cirq.CNOT(qreg_HVA[max(i,j)],qreg_HVA[min(i,j)]))
            HVA.append(cirq.H(qreg_HVA[i]))
            HVA.append(cirq.H(qreg_HVA[j]))
    #Construct Y-terms portion of layer
        HVA.append(cirq.ry(2*beta_mag[idx_layer]).on(q) for q in qreg_HVA)
        for (i,j) in yy_links:
            HVA.append(cirq.rx(math.pi/2).on(qreg_HVA[i]))
            HVA.append(cirq.rx(math.pi/2).on(qreg_HVA[j]))
            HVA.append(cirq.CNOT(qreg_HVA[max(i,j)],qreg_HVA[min(i,j)]))
        for (i,j) in yy_links_reversed:
            HVA.append(cirq.rz(2*beta[idx_layer]).on(qreg_HVA[min(i,j)]))
            HVA.append(cirq.CNOT(qreg_HVA[max(i,j)],qreg_HVA[min(i,j)]))
            HVA.append(cirq.rx(-math.pi/2).on(qreg_HVA[i]))
            HVA.append(cirq.rx(-math.pi/2).on(qreg_HVA[j]))
    #Construct Z-terms portion of layer
        HVA.append(cirq.rz(2*gamma_mag[idx_layer]).on(q) for q in qreg_HVA)
        for (i,j) in zz_links:
            HVA.append(cirq.CNOT(qreg_HVA[max(i,j)],qreg_HVA[min(i,j)]))
        for (i,j) in zz_links_reversed:
            HVA.append(cirq.rz(2*gamma[idx_layer]).on(qreg_HVA[min(i,j)]))
            HVA.append(cirq.CNOT(qreg_HVA[max(i,j)],qreg_HVA[min(i,j)]))
    return HVA

In [7]:
##Construct Objective Function
def objective(params, hamiltonian, qreg, simulator):
    ansatz = HVA_circuit(n_layers)
    params_dict = {'alpha_0': params[0], 'alpha_1': params[1], 'alpha_2': params[2], 'alpha_3': params[3], 'alpha_4': params[4], 'alpha_5': params[5], 'alpha_6': params[6], 'alpha_7': params[7], 'beta_0': params[8], 'beta_1': params[9], 'beta_2': params[10], 'beta_3': params[11], 'beta_4': params[12], 'beta_5': params[13], 'beta_6': params[14], 'beta_7': params[15], 'gamma_0': params[16], 'gamma_1': params[17], 'gamma_2': params[18], 'gamma_3': params[19], 'gamma_4': params[20], 'gamma_5': params[21], 'gamma_6': params[22], 'gamma_7': params[23], 'alpha_mag_0': params[24], 'alpha_mag_1': params[25], 'alpha_mag_2': params[26], 'alpha_mag_3': params[27], 'alpha_mag_4': params[28], 'alpha_mag_5': params[29], 'alpha_mag_6': params[30], 'alpha_mag_7': params[31], 'beta_mag_0': params[32], 'beta_mag_1': params[33], 'beta_mag_2': params[34], 'beta_mag_3': params[35], 'beta_mag_4': params[36], 'beta_mag_5': params[37], 'beta_mag_6': params[38], 'beta_mag_7': params[39], 'gamma_mag_0': params[40], 'gamma_mag_1': params[41], 'gamma_mag_2': params[42], 'gamma_mag_3': params[43], 'gamma_mag_4': params[44], 'gamma_mag_5': params[45], 'gamma_mag_6': params[46], 'gamma_mag_7': params[47]}
    expect = simulator.simulate_expectation_values(ansatz, hamiltonian, params_dict)
    return np.real(expect)    

In [12]:
##Perform VQE

#Initialize the simulator
gpu_options = qsimcirq.QSimOptions(use_gpu=True, gpu_mode=1)
simulator = qsimcirq.QSimSimulator(qsim_options=gpu_options)
qreg = cirq.NamedQubit.range(Nq, prefix="q")

#Specify number of layers
n_layers = 8

#Specify number of runs
#n_runs = 50

#Loop that executes VQE
#start_time = time.time()
#optimal_values = []
#cost_function_evals_collection = []
#initial_values_collection = []
#optimal_parameters_collection = []
#for run in range(n_runs):
    #x_init = np.random.uniform(-np.pi, np.pi, size=6*n_layers)
    #res = pybobyqa.solve(objective, x0=x_init, args=(Hamiltonian, qreg, simulator), maxfun = 5000)
    #optimal_values.append(res.f)
    #cost_function_evals_collection.append(res.nf)
    #initial_values_collection.append(x_init)
    #optimal_parameters_collection.append(res.x)
#end_time = time.time()
#print(f"n_layers: {n_layers}")
#print(f"Optimal Value: {min(optimal_values)}")
#print(f"Cost Function Evaluations: {cost_function_evals_collection[np.argmin(optimal_values)]}")
#print(f"Elapsed Time: {end_time - start_time}")
#print("*" * 40)

#1 Run of VQE
start_time = time.time()
res = pybobyqa.solve(objective, x0=np.random.uniform(-np.pi, np.pi, size=6*n_layers), args=(Hamiltonian, qreg, simulator), maxfun = 10000)
optimal_value = res.f
cost_function_evals = res.nf
optimal_parameters = res.x
end_time = time.time()
print(f"n_layers: {n_layers}")
print(f"Optimal Value: {optimal_value}")
print(f"Cost Function Evaluations: {cost_function_evals}")
print(f"Elapsed Time: {end_time - start_time}")
print("*" * 40)

n_layers: 8
Optimal Value: -14.591104895507646
Cost Function Evaluations: 2209
Elapsed Time: 1265.12371468544
****************************************


In [ ]:
#cuStateVec Results

#ED Result: -14.69101418686411

#n_layers: 1 (100 Initial Starting Points)
#Optimal Value: -14.53760226510414
#Cost Function Evaluations: 220
#Elapsed Time: 2668.075785636902
#****************************************


#n_layers: 2 (100 Initial Starting Points)
#Optimal Value: -14.615730287262227
#Cost Function Evaluations: 492
#Elapsed Time: 8578.33068394661
#****************************************


#n_layers: 3 (100 Initial Starting Points)
#Optimal Value: -14.618836603147018
#Cost Function Evaluations: 653
#Elapsed Time: 19203.95249581337
#****************************************

#n_layers: 4 (100 Initial Starting Points)
#Optimal Value: -14.631870723185942
#Cost Function Evaluations: 916
#Elapsed Time: 32530.648083925247
#****************************************

#n_layers: 5 (50 Initial Starting Points)
#Optimal Value: -14.614165076371565
#Cost Function Evaluations: 975
#Elapsed Time: 26435.226687669754
#****************************************

#n_layers: 8 (1 Initial Starting Point)
#Optimal Value: 
#Cost Function Evaluations: 
#Elapsed Time:
#****************************************

In [9]:
np.savetxt('24_Qubit_5_Layer_HVA_Initial_Points_GPU.txt', initial_values_collection)
np.savetxt('24_Qubit_5_Layer_HVA_Optimal_Values_GPU.txt', optimal_values)
np.savetxt('24_Qubit_5_Layer_HVA_Optimal_Parameters_GPU.txt', optimal_parameters_collection)
np.savetxt('24_Qubit_5_Layer_HVA_Cost_Function_Evaluations_GPU.txt', cost_function_evals_collection)